In [1]:
# 导入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import os

# 请求头 这里我尽量弄的全一点
headers = {
    'accept-encoding': 'gzip, deflate, br',
    'accept-language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cookie': 'pgv_pvid=7838988661; ptcz=a749c13713fc703b48592a49922470ed1eda1bcc7b68d2cf5be6fa79c731b76d; pgv_pvi=9718182912; RK=vkrweAJtMM; o_cookie=763074310; pac_uid=1_763074310; luin=o0763074310; lskey=000100007d08c463d9cc757cc4c5ae2cf3732897a65bf7de58e88b9524158f61b02e143bcf6124869366ea7d; tvfe_boss_uuid=e1c916f5e5e6e6bd; _txjk_whl_uuid_aa5wayli=58f307f1d8264ccb8aeb92b2f6225b14',
    'referer': 'https://news.qq.com/zt2020/page/feiyan.htm',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.61'
}

headers2 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Cookie': 'pgv_pvid=7838988661; ptcz=a749c13713fc703b48592a49922470ed1eda1bcc7b68d2cf5be6fa79c731b76d; pgv_pvi=9718182912; RK=vkrweAJtMM; o_cookie=763074310; pac_uid=1_763074310; luin=o0763074310; lskey=000100007d08c463d9cc757cc4c5ae2cf3732897a65bf7de58e88b9524158f61b02e143bcf6124869366ea7d; tvfe_boss_uuid=e1c916f5e5e6e6bd; _txjk_whl_uuid_aa5wayli=58f307f1d8264ccb8aeb92b2f6225b14; uin=o0763074310; skey=@xEkmEQkwn; qqmusic_uin=; qqmusic_key=; qqmusic_fromtag=; pgv_info=ssid=s1515695283',
    'Host': 'api.inews.qq.com',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.61'
}

In [2]:
# 先分析url
# https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery34102971291503109057_1594546996427&_=1594546996428
# https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery341023328877053019648_1594548638633&_=1594548638634
# https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=jQuery341010144709744391456_1594548760666&_=1594548760667
# 可以在看出最后的字段指的是时间戳 1594549307.2850437
url0 = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
res0 = requests.get(url0, headers=headers)
res0.json() # dict

None


In [4]:
data0 = json.loads(res0.json()['data']) # 因为data对应的值还是json字符串，还要转成Python字典
data0

{'lastUpdateTime': '2020-07-13 09:02:02',
 'chinaTotal': {'confirm': 85568,
  'heal': 80345,
  'dead': 4648,
  'nowConfirm': 575,
  'suspect': 7,
  'nowSevere': 3,
  'importedCase': 1979,
  'noInfect': 110},
 'chinaAdd': {'confirm': 46,
  'heal': 31,
  'dead': 0,
  'nowConfirm': 15,
  'suspect': 0,
  'nowSevere': 0,
  'importedCase': 8,
  'noInfect': 6},
 'isShowAdd': True,
 'showAddSwitch': {'all': True,
  'confirm': True,
  'suspect': True,
  'dead': True,
  'heal': True,
  'nowConfirm': True,
  'nowSevere': True,
  'importedCase': True,
  'noInfect': True},
 'areaTree': [{'name': '中国',
   'today': {'confirm': 46, 'isUpdated': True},
   'total': {'nowConfirm': 575,
    'confirm': 85568,
    'suspect': 7,
    'dead': 4648,
    'deadRate': '5.43',
    'showRate': False,
    'heal': 80345,
    'healRate': '93.90',
    'showHeal': True},
   'children': [{'name': '香港',
     'today': {'confirm': 38, 'confirmCuts': 0, 'isUpdated': True, 'tip': ''},
     'total': {'nowConfirm': 248,
      'c

# 国内数据爬取（包括省和具体市）

In [109]:
# 字段名   国/省/市名    现存确诊   累计确诊     疑似     死亡     死亡率     治愈    治愈率
# field_name = ['name', 'nowConfirm', 'confirm', 'suspect', 'dead', 'deadRate', 'heal', 'healRate']
# field_name_cn = ['地区', '现有确诊', '累计确诊', '疑似', '死亡', '死亡率', '治愈', '治愈率']
field_name_cn = {'name': '地区', 'nowConfirm': '现有确诊', 'confirm': '累计确诊', 'suspect': '疑似', 'dead': '死亡', 'deadRate': '死亡率', 'heal': '治愈', 'healRate': '治愈率'}

last_update_time = data['lastUpdateTime'].replace(':', '-') # 提取最后更新时间，并将里面的 : 替换成 - 因为文件名汇总不能有 :
filepath = f"./疫情数据/国内数据 {last_update_time}"
if not os.path.exists(filepath): # 先判断这个文件夹是否已存在，已存在说明数据时最新的
    os.mkdir(filepath) # 创建用最后更新时间来创建文件夹

china_data = {'name': data['areaTree'][0]['name']} # 新建一个字典，并地区名添加进去
china_data.update(data['areaTree'][0]['total']) # 合并地区的数据，字典合并字典
china_data = {k: [china_data[k]] for k in china_data.keys()} # 重构一下，将字段值用字典包裹，方便DataFrame直接合并

provinces_data =  pd.DataFrame()# 先创建一个总的数据集，后续数据往上添即可
provinces_data = pd.concat([provinces_data, pd.DataFrame(china_data)], axis=0, ignore_index=True) # 将全国的数据合并到总数据集中，ignore_index=True使得索引自增

# 各省份的数据都在 data['areaTree'][0]['children']这里，这是个列表，34个省级行政区
provinces = data['areaTree'][0]['children']
# 遍历这个列表取出每个省的数据和每个地区的数据
for i in provinces:
    # i['total'] 就是每个省的数据，里面的 showRate 和 showHeal 字段是不需要的 等同于这样data['areaTree'][0]['children'][0]['total']
    one_procevice_data = {'name': i['name']} # 将省份的名字添加进去
    one_procevice_data.update(i['total']) # 合并具体数据
    one_procevice_data = {k: [one_procevice_data[k]] for k in one_procevice_data.keys()} # 重构一下，将字段值用字典包裹，方便DataFrame直接合并
    provinces_data = pd.concat([provinces_data, pd.DataFrame(one_procevice_data)], axis=0, ignore_index=True) # 合并到总数据集中，ignore_index=True使得索引自增
    
    # 获取省内的各市的数据 省内的数据一次就保存到单独的csv文件中
    districts = i['children']
    districts_data = pd.DataFrame()
    districts_data = pd.concat([districts_data, pd.DataFrame(one_procevice_data)], axis=0, ignore_index=True) # 也把这个省总体的数据添加进去
    for j in districts:
        one_district_data = j['total'] # 是一个字典
        one_district_data['name'] = j['name'] # 将省份的名字添加进去
        one_district_data = {k: [one_district_data[k]] for k in one_district_data.keys()} # 重构一下，将字段值用字典包裹，方便DataFrame直接合并
        districts_data = pd.concat([districts_data, pd.DataFrame(one_district_data)], axis=0, ignore_index=True) # 合并到总数据集中，ignore_index=True使得索引自增
    
    # 保存这个省内各市的数据
    del districts_data['showRate'] # 删除这多余的两列
    del districts_data['showHeal']
    
    districts_data.rename(columns=field_name_cn, inplace = True) # 把列名改为中文
    districts_data.to_csv(f"{filepath}/{i['name']}.csv", encoding='GBK', index=False) # 保存到文件夹中 

# 保存每个省的数据
del provinces_data['showRate'] # 删除这多余的两列
del provinces_data['showHeal']

provinces_data.rename(columns=field_name_cn, inplace = True) # 把列名改为中文
provinces_data.to_csv(f"{filepath}/中国.csv", encoding='GBK', index=False) # 保存到文件夹中 

In [13]:
# 字段名   国/省/市名    现存确诊   累计确诊     疑似     死亡     死亡率     治愈    治愈率
# field_name = ['name', 'nowConfirm', 'confirm', 'suspect', 'dead', 'deadRate', 'heal', 'healRate']
# field_name_cn = ['地区', '现有确诊', '累计确诊', '疑似', '死亡', '死亡率', '治愈', '治愈率']
field_name_cn = {'name': '地区', 'nowConfirm': '现有确诊', 'confirm': '累计确诊', 'suspect': '疑似', 'dead': '死亡', 'deadRate': '死亡率', 'heal': '治愈', 'healRate': '治愈率'}

last_update_time = data0['lastUpdateTime'].replace(':', '-') # 提取最后更新时间，并将里面的 : 替换成 - 因为文件名汇总不能有 :
filepath = f"./疫情数据/国内数据 {last_update_time}"
if not os.path.exists(filepath): # 先判断这个文件夹是否已存在，已存在说明数据时最新的
    os.mkdir(filepath) # 创建用最后更新时间来创建文件夹

china_data = {'name': data0['areaTree'][0]['name']} # 新建一个字典，并地区名添加进去
china_data.update(data0['areaTree'][0]['total']) # 合并地区的数据，字典合并字典
provinces_data =  [china_data] # 先创建省份数据列表，后续数据直接追加尽量，列表中的数据都是字典，先把全国的数据添进去

# 各省份的数据都在 data['areaTree'][0]['children']这里，这是个列表，34个省级行政区
provinces = data0['areaTree'][0]['children']
# 遍历这个列表取出每个省的数据和每个地区的数据
for i in provinces:
    # i['total'] 就是每个省的数据，里面的 showRate 和 showHeal 字段是不需要的 等同于这样data['areaTree'][0]['children'][0]['total']
    one_procevice_data = {'name': i['name']} # 将省份的名字先添加进字典
    one_procevice_data.update(i['total']) # 合并具体数据 i['total']是字典型的数据
    provinces_data.append(one_procevice_data) # 向省份列表追加每个省的数据
    
    # 获取省内的各市的数据 省内的数据一次就保存到单独的csv文件中
    districts = i['children']
    districts_data = [one_procevice_data]
    for j in districts:
        one_district_data = {'name': j['name']}
        one_district_data.update(j['total'])
        districts_data.append(one_district_data)
        
    # 将字典构成的列表直接转为DataFrame
    districts_df = pd.DataFrame(districts_data)
    del districts_df['showRate'] # 删除这多余的两列
    del districts_df['showHeal']
    
    # 保存这个省内各市的数据
    districts_df.rename(columns=field_name_cn, inplace=True) # 把列名改为中文
    districts_df.to_csv(f"{filepath}/{i['name']}.csv", encoding='GBK', index=False) # 保存到文件夹中 

                        
# 将字典构成的列表直接转为DataFrame
provinces_df = pd.DataFrame(provinces_data)                        
del provinces_df['showRate'] # 删除这多余的两列
del provinces_df['showHeal']
                        
# 保存每个省的数据
provinces_df.rename(columns=field_name_cn, inplace=True) # 把列名改为中文
provinces_df.to_csv(f"{filepath}/中国.csv", encoding='GBK', index=False) # 保存到文件夹中 

In [ ]:
# 字段名   国/省/市名    现存确诊   累计确诊     疑似     死亡     死亡率     治愈    治愈率
# field_name = ['name', 'nowConfirm', 'confirm', 'suspect', 'dead', 'deadRate', 'heal', 'healRate']
# field_name_cn = ['地区', '现有确诊', '累计确诊', '疑似', '死亡', '死亡率', '治愈', '治愈率']
field_name_cn = {'name': '地区', 'nowConfirm': '现有确诊', 'confirm': '累计确诊', 'suspect': '疑似', 'dead': '死亡', 'deadRate': '死亡率', 'heal': '治愈', 'healRate': '治愈率'}

last_update_time = data0['lastUpdateTime'].replace(':', '-') # 提取最后更新时间，并将里面的 : 替换成 - 因为文件名汇总不能有 :
filepath = f"./疫情数据/国内数据 {last_update_time}"
if not os.path.exists(filepath): # 先判断这个文件夹是否已存在，已存在说明数据时最新的
    os.mkdir(filepath) # 创建用最后更新时间来创建文件夹

china_data = {'name': data0['areaTree'][0]['name']} # 新建一个字典，并地区名添加进去
china_data.update(data0['areaTree'][0]['total']) # 合并地区的数据，字典合并字典
provinces_data =  [china_data] # 先创建省份数据列表，后续数据直接追加尽量，列表中的数据都是字典，先把全国的数据添进去

# 各省份的数据都在 data['areaTree'][0]['children']这里，这是个列表，34个省级行政区
provinces = data0['areaTree'][0]['children']
# 遍历这个列表取出每个省的数据和每个地区的数据
for i in provinces:
    # i['total'] 就是每个省的数据，里面的 showRate 和 showHeal 字段是不需要的 等同于这样data['areaTree'][0]['children'][0]['total']
    one_procevice_data = {'name': i['name']} # 将省份的名字先添加进字典
    one_procevice_data.update(i['total']) # 合并具体数据 i['total']是字典型的数据
    provinces_data.append(one_procevice_data) # 向省份列表追加每个省的数据
    
    # 获取省内的各市的数据 省内的数据一次就保存到单独的csv文件中
    districts = i['children']
    districts_data = [one_procevice_data]
    for j in districts:
        one_district_data = {'name': j['name']}
        one_district_data.update(j['total'])
        districts_data.append(one_district_data)
        
    # 将字典构成的列表直接转为DataFrame
    districts_df = pd.DataFrame(districts_data)
    del districts_df['showRate'] # 删除这多余的两列
    del districts_df['showHeal']
    
    # 保存这个省内各市的数据
    districts_df.rename(columns=field_name_cn, inplace=True) # 把列名改为中文
    df_to_csv(districts_df, filepath+'/各省具体数据', f'i{"name"}.csv')
    # districts_df.to_csv(f"{filepath}/{i['name']}.csv", encoding='GBK', index=False) # 保存到文件夹中 

                        
# 将字典构成的列表直接转为DataFrame
provinces_df = pd.DataFrame(provinces_data)                        
del provinces_df['showRate'] # 删除这多余的两列
del provinces_df['showHeal']
                        
# 保存每个省的数据
provinces_df.rename(columns=field_name_cn, inplace=True) # 把列名改为中文
provinces_df.to_csv(f"{filepath}/中国.csv", encoding='GBK', index=False) # 保存到文件夹中 

In [91]:
data0['areaTree'][0]

{'name': '中国',
 'today': {'confirm': 73, 'isUpdated': True},
 'total': {'nowConfirm': 581,
  'confirm': 85560,
  'suspect': 7,
  'dead': 4648,
  'deadRate': '5.43',
  'showRate': False,
  'heal': 80331,
  'healRate': '93.89',
  'showHeal': True},
 'children': [{'name': '香港',
   'today': {'confirm': 66, 'confirmCuts': 0, 'isUpdated': True, 'tip': ''},
   'total': {'nowConfirm': 248,
    'confirm': 1469,
    'suspect': 0,
    'dead': 7,
    'deadRate': '0.48',
    'showRate': False,
    'heal': 1214,
    'healRate': '82.64',
    'showHeal': True},
   'children': [{'name': '地区待确认',
     'today': {'confirm': 66, 'confirmCuts': 0, 'isUpdated': True},
     'total': {'nowConfirm': 248,
      'confirm': 1469,
      'suspect': 0,
      'dead': 7,
      'deadRate': '0.48',
      'showRate': False,
      'heal': 1214,
      'healRate': '82.64',
      'showHeal': True}}]},
  {'name': '北京',
   'today': {'confirm': 0,
    'confirmCuts': 0,
    'isUpdated': True,
    'tip': '北京累计报告境外输入确诊病例174例。\n北京卫健

In [41]:
data0['areaTree'][0]['children']

[{'name': '香港',
  'today': {'confirm': 66, 'confirmCuts': 0, 'isUpdated': True, 'tip': ''},
  'total': {'nowConfirm': 248,
   'confirm': 1469,
   'suspect': 0,
   'dead': 7,
   'deadRate': '0.48',
   'showRate': False,
   'heal': 1214,
   'healRate': '82.64',
   'showHeal': True},
  'children': [{'name': '地区待确认',
    'today': {'confirm': 66, 'confirmCuts': 0, 'isUpdated': True},
    'total': {'nowConfirm': 248,
     'confirm': 1469,
     'suspect': 0,
     'dead': 7,
     'deadRate': '0.48',
     'showRate': False,
     'heal': 1214,
     'healRate': '82.64',
     'showHeal': True}}]},
 {'name': '北京',
  'today': {'confirm': 0,
   'confirmCuts': 0,
   'isUpdated': True,
   'tip': '北京累计报告境外输入确诊病例174例。\n北京卫健委未明确治愈病例和死亡病例的归属分区，北京分区现有确诊数据暂无法获取。'},
  'total': {'nowConfirm': 240,
   'confirm': 929,
   'suspect': 0,
   'dead': 9,
   'deadRate': '0.97',
   'showRate': False,
   'heal': 680,
   'healRate': '73.20',
   'showHeal': True},
  'children': [{'name': '丰台',
    'today': {'confirm': 0, '

In [79]:
# data['areaTree'][0]['total']
testdict0 = {i: [data0['areaTree'][0]['total'][i]] for i in data0['areaTree'][0]['total'].keys()}
testdict = {i: [data0['areaTree'][0]['children'][1]['total'][i]] for i in data0['areaTree'][0]['children'][1]['total'].keys()}
testdict

{'nowConfirm': [240],
 'confirm': [929],
 'suspect': [0],
 'dead': [9],
 'deadRate': ['0.97'],
 'showRate': [False],
 'heal': [680],
 'healRate': ['73.20'],
 'showHeal': [True]}

In [89]:
testpd = pd.DataFrame()
testpd = pd.concat([testpd, pd.DataFrame(testdict0)], axis=0,  ignore_index=True)
testpd = pd.concat([testpd, pd.DataFrame(testdict)], axis=0,  ignore_index=True)
testpd

,,


# 国内随时间变化的数据爬取

In [69]:
url1 = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_other'
res1 = requests.get(url1, headers=headers)
res1.json() # dict

{'ret': 0,
 'data': '{"chinaDayList":[{"confirm":41,"suspect":0,"dead":1,"heal":0,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"2.4","healRate":"0.0","date":"01.13","noInfect":0},{"confirm":41,"suspect":0,"dead":1,"heal":0,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"2.4","healRate":"0.0","date":"01.14","noInfect":0},{"confirm":41,"suspect":0,"dead":2,"heal":5,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"4.9","healRate":"12.2","date":"01.15","noInfect":0},{"confirm":45,"suspect":0,"dead":2,"heal":8,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"4.4","healRate":"17.8","date":"01.16","noInfect":0},{"confirm":62,"suspect":0,"dead":2,"heal":12,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"3.2","healRate":"19.4","date":"01.17","noInfect":0},{"confirm":198,"suspect":0,"dead":3,"heal":17,"nowConfirm":0,"nowSevere":0,"importedCase":0,"deadRate":"1.5","healRate":"8.6","date":"01.18","noInfect":0},{"confirm":275,"suspect":0,"dead":4,"h

In [70]:
data1 = json.loads(res1.json()['data']) # 因为data对应的值还是json字符串，还要转成Python字典
data1

{'chinaDayList': [{'confirm': 41,
   'suspect': 0,
   'dead': 1,
   'heal': 0,
   'nowConfirm': 0,
   'nowSevere': 0,
   'importedCase': 0,
   'deadRate': '2.4',
   'healRate': '0.0',
   'date': '01.13',
   'noInfect': 0},
  {'confirm': 41,
   'suspect': 0,
   'dead': 1,
   'heal': 0,
   'nowConfirm': 0,
   'nowSevere': 0,
   'importedCase': 0,
   'deadRate': '2.4',
   'healRate': '0.0',
   'date': '01.14',
   'noInfect': 0},
  {'confirm': 41,
   'suspect': 0,
   'dead': 2,
   'heal': 5,
   'nowConfirm': 0,
   'nowSevere': 0,
   'importedCase': 0,
   'deadRate': '4.9',
   'healRate': '12.2',
   'date': '01.15',
   'noInfect': 0},
  {'confirm': 45,
   'suspect': 0,
   'dead': 2,
   'heal': 8,
   'nowConfirm': 0,
   'nowSevere': 0,
   'importedCase': 0,
   'deadRate': '4.4',
   'healRate': '17.8',
   'date': '01.16',
   'noInfect': 0},
  {'confirm': 62,
   'suspect': 0,
   'dead': 2,
   'heal': 12,
   'nowConfirm': 0,
   'nowSevere': 0,
   'importedCase': 0,
   'deadRate': '3.2',
   'hea

In [71]:
data1.keys()

dict_keys(['chinaDayList', 'chinaDayAddList', 'dailyNewAddHistory', 'dailyHistory', 'wuhanDayList', 'articleList', 'provinceCompare', 'cityStatis', 'nowConfirmStatis'])

In [81]:
pd.DataFrame(data1['chinaDayList']) # 国内疫情新增数据随时间变化

,confirm,suspect,dead,heal,nowConfirm,nowSevere,importedCase,deadRate,healRate,date,noInfect
0,41,0,1,0,0,0,0,2.4,0.0,01.13,0
1,41,0,1,0,0,0,0,2.4,0.0,01.14,0
2,41,0,2,5,0,0,0,4.9,12.2,01.15,0
3,45,0,2,8,0,0,0,4.4,17.8,01.16,0
4,62,0,2,12,0,0,0,3.2,19.4,01.17,0
...,...,...,...,...,...,...,...,...,...,...,...
177,85399,5,4648,80240,511,5,1958,5.4,94.0,07.08,112
178,85445,8,4648,80268,529,4,1962,5.4,93.9,07.09,113
179,85487,8,4648,80293,546,3,1964,5.4,93.9,07.10,111
180,85522,7,4648,80314,560,3,1971,5.4,93.9,07.11,112


In [84]:
pd.DataFrame(data1['chinaDayAddList']) # 国内疫情数据随时间变化

,confirm,suspect,dead,heal,importedCase,infect,deadRate,healRate,date
0,77,27,0,0,0,0,0.0,0.0,01.20
1,149,53,3,0,0,0,2.0,0.0,01.21
2,131,257,8,0,0,0,6.1,0.0,01.22
3,259,680,8,6,0,0,3.1,2.3,01.23
4,444,1118,16,3,0,0,3.6,0.7,01.24
...,...,...,...,...,...,...,...,...,...
171,46,3,0,28,4,3,0.0,60.9,07.09
172,42,0,0,25,2,4,0.0,59.5,07.10
173,35,0,0,21,7,5,0.0,60.0,07.11
174,46,0,0,31,8,6,0.0,67.4,07.12


In [87]:
pd.DataFrame(data1['dailyNewAddHistory']) # 历史每日新增数据（湖北、国家、和非湖北）

,date,hubei,country,notHubei
0,01.20,72,77,5
1,01.21,105,149,44
2,01.22,69,131,62
3,01.23,105,259,154
4,01.24,180,444,264
...,...,...,...,...
170,07.08,0,33,33
171,07.09,0,46,46
172,07.10,0,42,42
173,07.11,0,35,35


In [90]:
data1['dailyHistory'] # 历史每天疫情数据（湖北、国家、和非湖北） 这个数据补搞了

[{'date': '01.20',
  'hubei': {'dead': 6,
   'heal': 25,
   'nowConfirm': 239,
   'deadRate': '2.22',
   'healRate': '9.26'},
  'notHubei': {'dead': 0,
   'heal': 0,
   'nowConfirm': 52,
   'deadRate': '0.00',
   'healRate': '0.00'},
  'country': {'dead': 6,
   'heal': 25,
   'nowConfirm': 291,
   'deadRate': '2.06',
   'healRate': '8.59'}},
 {'date': '01.23',
  'hubei': {'dead': 24,
   'heal': 31,
   'nowConfirm': 494,
   'deadRate': '4.37',
   'healRate': '5.65'},
  'notHubei': {'dead': 1,
   'heal': 3,
   'nowConfirm': 282,
   'deadRate': '0.35',
   'healRate': '1.05'},
  'country': {'dead': 25,
   'heal': 34,
   'nowConfirm': 776,
   'deadRate': '2.99',
   'healRate': '4.07'}},
 {'date': '01.24',
  'hubei': {'dead': 39,
   'heal': 32,
   'nowConfirm': 658,
   'deadRate': '5.35',
   'healRate': '4.39'},
  'notHubei': {'dead': 2,
   'heal': 6,
   'nowConfirm': 560,
   'deadRate': '0.35',
   'healRate': '1.06'},
  'country': {'dead': 41,
   'heal': 38,
   'nowConfirm': 1218,
   'deadR

In [106]:
province_compare = pd.DataFrame(data1['provinceCompare']) # 各省数据比较（现存确诊，新增确诊，死亡，治愈，连续0增天数
province_compare = pd.DataFrame(province_compare.values.T, index=province_compare.columns, columns=province_compare.index) # 转置
province_compare

,nowConfirm,confirmAdd,dead,heal,zero
上海,0,1,0,1,0
云南,0,0,0,0,5
内蒙古,4,4,0,0,0
北京,-14,0,0,14,7
台湾,0,0,0,0,2
吉林,0,0,0,0,50
四川,0,0,0,0,4
天津,0,0,0,0,1
宁夏,0,0,0,0,131
安徽,0,0,0,0,95


In [107]:
# 存储数据
df_to_csv(pd.DataFrame(data1['chinaDayList']), filepath, '国内疫情数据随时间变化.csv')
df_to_csv(pd.DataFrame(data1['chinaDayAddList']), filepath, '国内疫情新增数据随时间变化.csv')
df_to_csv(pd.DataFrame(data1['dailyNewAddHistory']), filepath, '历史每日新增数据（湖北、国家、和非湖北）.csv')
df_to_csv(province_compare, filepath, '各省今天数据比较.csv', index=True) # 保留索引

# 外国总体数据的爬取（按国区分）

In [30]:
# 先判断外国的文件夹在不在，不存在就创建，文件名的更新时间用与上面一致的时间
filepath2 =  f"./疫情数据/外国数据 {last_update_time}"
if not os.path.exists(filepath2): # 先判断这个文件夹是否已存在，已存在说明数据时最新的
    os.mkdir(filepath2) # 创建用最后更新时间来创建文件夹

In [15]:
url2 = 'https://api.inews.qq.com/newsqa/v1/automation/modules/list?modules=FAutoCountryWeekCompRank,FAutoContinentConfirmStatis,FAutoConfirmMillionRankList,FAutoHealDeadRateRankList'
res2 = requests.get(url2, headers=headers)
res2.json()

{'ret': 0,
 'info': '',
 'data': {'FAutoConfirmMillionRankList': [{'nation': '卡塔尔',
    'confirm': 103598,
    'population': 2781677,
    'rate': 37243},
   {'nation': '巴林', 'confirm': 32941, 'population': 1569439, 'rate': 20989},
   {'nation': '智利', 'confirm': 315041, 'population': 18729160, 'rate': 16821},
   {'nation': '科威特', 'confirm': 54894, 'population': 4137309, 'rate': 13268},
   {'nation': '阿曼', 'confirm': 56015, 'population': 4829483, 'rate': 11599},
   {'nation': '巴拿马', 'confirm': 45633, 'population': 4176873, 'rate': 10925},
   {'nation': '亚美尼亚', 'confirm': 31969, 'population': 2951776, 'rate': 10830},
   {'nation': '美国',
    'confirm': 3413995,
    'population': 327167434,
    'rate': 10435},
   {'nation': '秘鲁', 'confirm': 326326, 'population': 31989256, 'rate': 10201},
   {'nation': '巴西', 'confirm': 1864681, 'population': 209469333, 'rate': 8902},
   {'nation': '新加坡', 'confirm': 45961, 'population': 5638676, 'rate': 8151},
   {'nation': '卢森堡', 'confirm': 4925, 'population

In [16]:
data2 = res2.json()['data']
data2

{'FAutoConfirmMillionRankList': [{'nation': '卡塔尔',
   'confirm': 103598,
   'population': 2781677,
   'rate': 37243},
  {'nation': '巴林', 'confirm': 32941, 'population': 1569439, 'rate': 20989},
  {'nation': '智利', 'confirm': 315041, 'population': 18729160, 'rate': 16821},
  {'nation': '科威特', 'confirm': 54894, 'population': 4137309, 'rate': 13268},
  {'nation': '阿曼', 'confirm': 56015, 'population': 4829483, 'rate': 11599},
  {'nation': '巴拿马', 'confirm': 45633, 'population': 4176873, 'rate': 10925},
  {'nation': '亚美尼亚', 'confirm': 31969, 'population': 2951776, 'rate': 10830},
  {'nation': '美国', 'confirm': 3413995, 'population': 327167434, 'rate': 10435},
  {'nation': '秘鲁', 'confirm': 326326, 'population': 31989256, 'rate': 10201},
  {'nation': '巴西', 'confirm': 1864681, 'population': 209469333, 'rate': 8902},
  {'nation': '新加坡', 'confirm': 45961, 'population': 5638676, 'rate': 8151},
  {'nation': '卢森堡', 'confirm': 4925, 'population': 607728, 'rate': 8104},
  {'nation': '瑞典', 'confirm': 748

In [35]:
# 定义一个存储DataFrame的函数
def df_to_csv(df=None, filepath=None, filename=None, encoding='GBK', index=False):
    if not os.path.exists(filepath): # 先判断这个文件夹是否已存在，已存在说明数据时最新的
        os.mkdir(filepath) # 创建用最后更新时间来创建文件夹
    df.to_csv(filepath + '/' + filename, encoding=encoding, index=index) # 保存到文件夹中

In [36]:
# 存储数据直接用 二次包装的to_csv方法 这里并没有修改列名为中文
df_to_csv(pd.DataFrame(data2['FAutoConfirmMillionRankList']), filepath2, '外国每百万人确诊数.csv')
df_to_csv(pd.DataFrame(data2['FAutoContinentConfirmStatis']), filepath2, '外国累计确诊周增幅.csv')
df_to_csv(pd.DataFrame(data2['FAutoCountryWeekCompRank']), filepath2, '外国累计确诊七日增幅.csv')
df_to_csv(pd.DataFrame(data2['FAutoHealDeadRateRankList']['deadHead']), filepath2, '外国死亡率前10名.csv')
df_to_csv(pd.DataFrame(data2['FAutoHealDeadRateRankList']['deadTail']), filepath2, '外国死亡率后10名.csv')
df_to_csv(pd.DataFrame(data2['FAutoHealDeadRateRankList']['healHead']), filepath2, '外国治愈前10名.csv')
df_to_csv(pd.DataFrame(data2['FAutoHealDeadRateRankList']['healTail']), filepath2, '外国治愈后10名.csv')

In [17]:
data2['FAutoConfirmMillionRankList']

[{'nation': '卡塔尔', 'confirm': 103598, 'population': 2781677, 'rate': 37243},
 {'nation': '巴林', 'confirm': 32941, 'population': 1569439, 'rate': 20989},
 {'nation': '智利', 'confirm': 315041, 'population': 18729160, 'rate': 16821},
 {'nation': '科威特', 'confirm': 54894, 'population': 4137309, 'rate': 13268},
 {'nation': '阿曼', 'confirm': 56015, 'population': 4829483, 'rate': 11599},
 {'nation': '巴拿马', 'confirm': 45633, 'population': 4176873, 'rate': 10925},
 {'nation': '亚美尼亚', 'confirm': 31969, 'population': 2951776, 'rate': 10830},
 {'nation': '美国', 'confirm': 3413995, 'population': 327167434, 'rate': 10435},
 {'nation': '秘鲁', 'confirm': 326326, 'population': 31989256, 'rate': 10201},
 {'nation': '巴西', 'confirm': 1864681, 'population': 209469333, 'rate': 8902},
 {'nation': '新加坡', 'confirm': 45961, 'population': 5638676, 'rate': 8151},
 {'nation': '卢森堡', 'confirm': 4925, 'population': 607728, 'rate': 8104},
 {'nation': '瑞典', 'confirm': 74898, 'population': 10183175, 'rate': 7355},
 {'nation'

In [18]:
pd.DataFrame(data2['FAutoConfirmMillionRankList']) # 外国每百万人确诊数

,nation,confirm,population,rate
0,卡塔尔,103598,2781677,37243
1,巴林,32941,1569439,20989
2,智利,315041,18729160,16821
3,科威特,54894,4137309,13268
4,阿曼,56015,4829483,11599
...,...,...,...,...
127,也门,1465,28498687,51
128,尼日尔,1099,22442948,49
129,泰国,3217,69428524,46
130,莫桑比克,1157,29495962,39


In [21]:
pd.DataFrame(data2['FAutoContinentConfirmStatis']) # 外国累计确诊周增幅

,date,confirm,rate,range
0,02/16,687,0,
1,02/23,1895,175,02/17-02/23
2,03/01,7342,287,02/24-03/01
3,03/08,26959,267,03/02-03/08
4,03/15,79559,195,03/09-03/15
5,03/22,235306,195,03/16-03/22
6,03/29,600127,155,03/23-03/29
7,04/05,1143421,90,03/30-04/05
8,04/12,1716850,50,04/06-04/12
9,04/19,2271603,32,04/13-04/19


In [22]:
pd.DataFrame(data2['FAutoCountryWeekCompRank']) # 外国累计确诊七日增幅 即最近一周增长复读最快的10个国家

,nation,day,day7,rate
0,黑山,1221,781,56.34
1,哥斯达黎加,7596,4996,52.04
2,马达加斯加,4867,3250,49.75
3,巴勒斯坦,6230,4341,43.52
4,南非,276242,196750,40.40
5,吉尔吉斯斯坦,10629,7691,38.20
6,津巴布韦,985,716,37.57
7,利比亚,1433,1046,37.00
8,斯威士兰,1351,988,36.74
9,马拉维,2364,1742,35.71


In [25]:
pd.DataFrame(data2['FAutoHealDeadRateRankList']['deadHead']) # 外国死亡率前10名

,country,confirm,heal,dead,healRate,deadRate
0,也门,1465,659,417,44.98,28.46
1,比利时,62606,17196,9782,27.47,15.62
2,英国,291154,1378,44904,0.47,15.42
3,法国,208015,78513,30007,37.74,14.43
4,意大利,243061,194928,34954,80.20,14.38
5,匈牙利,4234,3036,595,71.71,14.05
6,荷兰,51237,193,6156,0.38,12.01
7,墨西哥,295268,229856,34730,77.85,11.76
8,西班牙,300988,196958,28403,65.44,9.44
9,加拿大,109348,72954,8829,66.72,8.07


In [26]:
pd.DataFrame(data2['FAutoHealDeadRateRankList']['deadTail']) # 外国死亡率后10名

,country,confirm,heal,dead,healRate,deadRate
0,乌干达,1025,972,0,94.83,0.00
1,新加坡,45961,42285,26,92.00,0.06
2,卡塔尔,103598,99743,147,96.28,0.14
3,尼泊尔,16801,8589,38,51.12,0.23
4,卢旺达,1337,684,4,51.16,0.30
5,巴林,32941,28425,108,86.29,0.33
6,哥斯达黎加,7596,2239,30,29.48,0.39
7,斯里兰卡,2617,1981,11,75.70,0.42
8,乌兹别克斯坦,12997,7852,60,60.41,0.46
9,阿曼,56015,36098,257,64.44,0.46


In [27]:
pd.DataFrame(data2['FAutoHealDeadRateRankList']['healHead'])  # 外国治愈前10名

,country,confirm,heal,dead,healRate,deadRate
0,冰岛,1896,1865,10,98.36,0.53
1,马来西亚,8718,8519,122,97.72,1.40
2,新西兰,1544,1497,22,96.96,1.42
3,卡塔尔,103598,99743,147,96.28,0.14
4,泰国,3217,3088,58,95.99,1.80
5,乌干达,1025,972,0,94.83,0.00
6,吉布提,4972,4712,56,94.77,1.13
7,爱沙尼亚,2014,1895,69,94.09,3.43
8,丹麦,13147,12278,609,93.39,4.63
9,芬兰,7294,6800,329,93.23,4.51


In [29]:
pd.DataFrame(data2['FAutoHealDeadRateRankList']['healTail'])  # 外国治愈后10名

,country,confirm,heal,dead,healRate,deadRate
0,荷兰,51237,193,6156,0.38,12.01
1,英国,291154,1378,44904,0.47,15.42
2,瑞典,74898,4971,5526,6.64,7.38
3,洪都拉斯,27583,2901,771,10.52,2.80
4,危地马拉,29355,4214,1219,14.36,4.15
5,巴勒斯坦,6230,942,36,15.12,0.58
6,马拉维,2364,557,38,23.56,1.61
7,利比亚,1433,341,39,23.80,2.72
8,菲律宾,54222,14037,1372,25.89,2.53
9,黑山,1221,325,23,26.62,1.88


# 外国各国具体数据的爬取（有的国家包括其具体行政区的数据）

In [37]:
url3 = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_foreign'
res3 = requests.get(url3, headers=headers)
res3.json()

{'ret': 0,
 'data': '{"foreignList":[{"name":"美国","continent":"北美洲","date":"07.13","isUpdated":true,"confirmAdd":58349,"confirmAddCut":0,"confirm":3413995,"suspect":0,"dead":137782,"heal":1517084,"nowConfirm":1759129,"confirmCompare":58349,"nowConfirmCompare":30305,"healCompare":27664,"deadCompare":380,"children":[{"name":"纽约","date":"07.12","nameMap":"New York","isUpdated":false,"confirmAdd":0,"confirmAddCut":0,"confirm":406328,"suspect":0,"dead":32343,"heal":57171},{"name":"加利福尼亚","date":"07.12","nameMap":"California","isUpdated":false,"confirmAdd":0,"confirmAddCut":0,"confirm":318761,"suspect":0,"dead":7029,"heal":80249},{"name":"佛罗里达","date":"07.12","nameMap":"Florida","isUpdated":false,"confirmAdd":0,"confirmAddCut":0,"confirm":254511,"suspect":0,"dead":4197,"heal":381},{"name":"德克萨斯","date":"07.12","nameMap":"Texas","isUpdated":false,"confirmAdd":0,"confirmAddCut":0,"confirm":254415,"suspect":0,"dead":3138,"heal":127880},{"name":"新泽西","date":"07.12","nameMap":"New Jersey","isUpda

In [40]:
data3 = json.loads(res3.json()['data']) # 因为data对应的值还是json字符串，还要转成Python字典
data3

{'foreignList': [{'name': '美国',
   'continent': '北美洲',
   'date': '07.13',
   'isUpdated': True,
   'confirmAdd': 58349,
   'confirmAddCut': 0,
   'confirm': 3413995,
   'suspect': 0,
   'dead': 137782,
   'heal': 1517084,
   'nowConfirm': 1759129,
   'confirmCompare': 58349,
   'nowConfirmCompare': 30305,
   'healCompare': 27664,
   'deadCompare': 380,
   'children': [{'name': '纽约',
     'date': '07.12',
     'nameMap': 'New York',
     'isUpdated': False,
     'confirmAdd': 0,
     'confirmAddCut': 0,
     'confirm': 406328,
     'suspect': 0,
     'dead': 32343,
     'heal': 57171},
    {'name': '加利福尼亚',
     'date': '07.12',
     'nameMap': 'California',
     'isUpdated': False,
     'confirmAdd': 0,
     'confirmAddCut': 0,
     'confirm': 318761,
     'suspect': 0,
     'dead': 7029,
     'heal': 80249},
    {'name': '佛罗里达',
     'date': '07.12',
     'nameMap': 'Florida',
     'isUpdated': False,
     'confirmAdd': 0,
     'confirmAddCut': 0,
     'confirm': 254511,
     'suspec

In [41]:
data3.keys()

dict_keys(['foreignList', 'globalStatis', 'globalDailyHistory', 'importStatis', 'countryAddConfirmRankList', 'countryConfirmWeekCompareRankList', 'continentStatis'])

In [47]:
data3['foreignList']

[{'name': '美国',
  'continent': '北美洲',
  'date': '07.13',
  'isUpdated': True,
  'confirmAdd': 58349,
  'confirmAddCut': 0,
  'confirm': 3413995,
  'suspect': 0,
  'dead': 137782,
  'heal': 1517084,
  'nowConfirm': 1759129,
  'confirmCompare': 58349,
  'nowConfirmCompare': 30305,
  'healCompare': 27664,
  'deadCompare': 380,
  'children': [{'name': '纽约',
    'date': '07.12',
    'nameMap': 'New York',
    'isUpdated': False,
    'confirmAdd': 0,
    'confirmAddCut': 0,
    'confirm': 406328,
    'suspect': 0,
    'dead': 32343,
    'heal': 57171},
   {'name': '加利福尼亚',
    'date': '07.12',
    'nameMap': 'California',
    'isUpdated': False,
    'confirmAdd': 0,
    'confirmAddCut': 0,
    'confirm': 318761,
    'suspect': 0,
    'dead': 7029,
    'heal': 80249},
   {'name': '佛罗里达',
    'date': '07.12',
    'nameMap': 'Florida',
    'isUpdated': False,
    'confirmAdd': 0,
    'confirmAddCut': 0,
    'confirm': 254511,
    'suspect': 0,
    'dead': 4197,
    'heal': 381},
   {'name': '德克

In [56]:
data3['foreignList'][0]

{'name': '美国',
 'continent': '北美洲',
 'date': '07.13',
 'isUpdated': True,
 'confirmAdd': 58349,
 'confirmAddCut': 0,
 'confirm': 3413995,
 'suspect': 0,
 'dead': 137782,
 'heal': 1517084,
 'nowConfirm': 1759129,
 'confirmCompare': 58349,
 'nowConfirmCompare': 30305,
 'healCompare': 27664,
 'deadCompare': 380,
 'children': [{'name': '纽约',
   'date': '07.12',
   'nameMap': 'New York',
   'isUpdated': False,
   'confirmAdd': 0,
   'confirmAddCut': 0,
   'confirm': 406328,
   'suspect': 0,
   'dead': 32343,
   'heal': 57171},
  {'name': '加利福尼亚',
   'date': '07.12',
   'nameMap': 'California',
   'isUpdated': False,
   'confirmAdd': 0,
   'confirmAddCut': 0,
   'confirm': 318761,
   'suspect': 0,
   'dead': 7029,
   'heal': 80249},
  {'name': '佛罗里达',
   'date': '07.12',
   'nameMap': 'Florida',
   'isUpdated': False,
   'confirmAdd': 0,
   'confirmAddCut': 0,
   'confirm': 254511,
   'suspect': 0,
   'dead': 4197,
   'heal': 381},
  {'name': '德克萨斯',
   'date': '07.12',
   'nameMap': 'Texas'

In [64]:
data3['foreignList'][5]

{'name': '伊朗',
 'continent': '亚洲',
 'date': '03.31',
 'isUpdated': False,
 'confirmAdd': 0,
 'confirmAddCut': 0,
 'confirm': 44606,
 'suspect': 0,
 'dead': 2898,
 'heal': 14656,
 'nowConfirm': 27052,
 'confirmCompare': 0,
 'nowConfirmCompare': 0,
 'healCompare': 0,
 'deadCompare': 0}

In [59]:
# field_name_cn2 = {'name': '国家', 'nowConfirm': '现有确诊', 'confirm': '累计确诊', 'suspect': '疑似', 'dead': '死亡', 'deadRate': '死亡率', 'heal': '治愈', 'healRate': '治愈率'}

foreign_data = [] # 新建一个字典，用于存储每个外国的

# 各国家的数据
countries = data3['foreignList']
# 遍历这个列表取出每个省的数据和每个地区的数据
for i in countries:
    # 先判断这个国家有没有具体类似我国省这样的行政区的数据，即判断有没有 children 这个字段
    if 'children' in i: # 等同于 in i.keys()
        # 当存在时，也爬取行政区的数据
        # 用pop()把children字段的值弹出来，这样原数据就和其他没有该字段的数据类型一样了
        # 但是需要注意的是，由于字典是引用型的数据，所有一旦pop()就会改变最本源的数据，就是无法执行第二次
        region_data = i.pop('children') 
        # 由于数据直接是列表套字典的形式，直接就可以转成DataFrame
        # 不过这里也把这个国家总体的数据插入到顶部，方便整体观察
        region_data.insert(0, i) # 现在的i已经没有children字段了
        # 转换成DataFrame并存储到本地
        df_to_csv(pd.DataFrame(region_data), filepath=filepath2, filename=f"{i['name']}.csv")
    
    # 同时把改过总体数据存储到记录外国总体数据的列表中
    foreign_data.append(i)

# 遍历完后同上没存储操作一样
df_to_csv(pd.DataFrame(foreign_data), filepath=filepath2, filename="外国疫情总体数据.csv") 
                  
# 通过检查后，该数据没用

In [51]:
data3['foreignList'][0]['children'][-1]

{'name': '地区待确认',
 'date': '04.02',
 'nameMap': '',
 'isUpdated': False,
 'confirmAdd': 0,
 'confirmAddCut': 0,
 'confirm': 0,
 'suspect': 0,
 'dead': 0,
 'heal': 0}

In [42]:
data3['globalStatis'] # 外国确诊数据

{'nowConfirm': 4802545,
 'confirm': 12937621,
 'heal': 7568978,
 'dead': 566098,
 'nowConfirmAdd': 45300,
 'confirmAdd': 125844,
 'healAdd': 78132,
 'deadAdd': 2412,
 'lastUpdateTime': '2020-07-13 10:19:36'}

In [111]:
# data3['globalDailyHistory'] # 全球每日数据 假
global_daily_data = [] # 存储全球每日数据的集合
for i in data3['globalDailyHistory']:
    day_data = {'date': i['date']}
    day_data.update(i['all']) # 就all字段中的每日数据追加进去
    global_daily_data.append(day_data)
# pd.DataFrame(global_daily_data) 
df_to_csv(pd.DataFrame(global_daily_data), filepath2, '全球每日数据.csv') # 转换成DataFrame，并存储到本地

In [112]:
data3['importStatis']['TopList'] # 输入病例
pd.DataFrame(data3['importStatis']['TopList'])

,province,importedCase
0,黑龙江,386
1,上海,383
2,广东,255
3,北京,174
4,内蒙古,172
5,陕西,75
6,甘肃,75
7,福建,67
8,山西,66
9,天津,66


In [66]:
data3['countryAddConfirmRankList'] # 外国24小时新增确诊前10名
pd.DataFrame(data3['countryAddConfirmRankList'])

,nation,addConfirm
0,美国,66281
1,巴西,45048
2,印度,28637
3,南非,13497
4,墨西哥,6891
5,哥伦比亚,6803
6,俄罗斯,6615
7,阿根廷,3367
8,秘鲁,3198
9,沙特阿拉伯,2994


In [67]:
data3['countryConfirmWeekCompareRankList'] # 外国七天变化排名
pd.DataFrame(data3['countryConfirmWeekCompareRankList'])

,nation,day,day7,rate
0,日本本土,4257,2178,95.45
1,马达加斯加,4578,2728,67.82
2,黑山,1164,720,61.67
3,哥斯达黎加,6845,4311,58.78
4,被占巴勒斯坦领土,6688,4250,57.36
5,利比亚,1389,918,51.31
6,吉尔吉斯斯坦,10629,7094,49.83
7,科索沃,4672,3313,41.02
8,津巴布韦,982,698,40.69
9,南非,264184,187977,40.54


In [68]:
data3['continentStatis'] # 各大洲数据随时间变化趋势
pd.DataFrame(data3['continentStatis'])

,date,statis,nowConfirm,rate,range
0,02/16,"{'亚洲': 244, '其他': 355, '北美洲': 23, '大洋洲': 15, '...",595,0,
1,02/23,"{'亚洲': 960, '其他': 691, '北美洲': 42, '大洋洲': 17, '...",1716,188,02/17-02/23
2,03/01,"{'亚洲': 4238, '其他': 712, '北美洲': 94, '南美洲': 3, '...",5684,231,02/24-03/01
3,03/08,"{'亚洲': 15143, '其他': 696, '北美洲': 528, '南美洲': 55...",23184,307,03/02-03/08
4,03/15,"{'亚洲': 25683, '其他': 712, '北美洲': 3315, '南美洲': 3...",67490,191,03/09-03/15
5,03/22,"{'亚洲': 41052, '其他': 712, '北美洲': 29020, '南美洲': ...",200985,197,03/16-03/22
6,03/29,"{'亚洲': 75791, '其他': 712, '北美洲': 133037, '南美洲':...",500313,148,03/23-03/29
7,04/05,{'亚洲': 584},547,-99,03/30-04/05
8,04/19,"{'北美洲': 735366, '欧洲': 4465}",633670,115744,04/06-04/19
9,04/26,"{'北美洲': 941628, '欧洲': 4465}",786034,24,04/20-04/26


In [ ]:
# 存储数据
df_to_csv(pd.DataFrame(data3['importStatis']['TopList']), filepath, '输入病例前十省.csv')
df_to_csv(pd.DataFrame(data3['countryAddConfirmRankList']), filepath2, '外国24小时新增确诊前10名.csv')
df_to_csv(pd.DataFrame(data3['countryConfirmWeekCompareRankList']), filepath2, '外国七天变化排名.csv')

# 按各大洲的数据爬取和疫情到现在随时间变化的数据

In [114]:
url4 = "https://api.inews.qq.com/newsqa/v1/automation/modules/list?modules=FAutoGlobalStatis,FAutoContinentStatis,FAutoGlobalDailyList,FAutoCountryConfirmAdd"
res4 = requests.get(url4, headers=headers2)
data4 = res4.json()['data']
data4

{'FAutoContinentStatis': [{'date': '02/16',
   'statis': {'亚洲': 247, '其他': 355, '北美洲': 23, '大洋洲': 15, '欧洲': 47},
   'nowConfirm': 600,
   'rate': 0,
   'range': ''},
  {'date': '02/23',
   'statis': {'亚洲': 969, '其他': 691, '北美洲': 42, '大洋洲': 17, '欧洲': 176},
   'nowConfirm': 1718,
   'rate': 186,
   'range': '02/17-02/23'},
  {'date': '03/01',
   'statis': {'亚洲': 4938,
    '其他': 712,
    '北美洲': 94,
    '南美洲': 3,
    '大洋洲': 29,
    '欧洲': 1564,
    '非洲': 2},
   'nowConfirm': 6210,
   'rate': 261,
   'range': '02/24-03/01'},
  {'date': '03/08',
   'statis': {'亚洲': 15145,
    '其他': 696,
    '北美洲': 528,
    '南美洲': 55,
    '大洋洲': 81,
    '欧洲': 10373,
    '非洲': 81},
   'nowConfirm': 23190,
   'rate': 273,
   'range': '03/02-03/08'},
  {'date': '03/15',
   'statis': {'亚洲': 25684,
    '其他': 712,
    '北美洲': 3315,
    '南美洲': 325,
    '大洋洲': 280,
    '欧洲': 48928,
    '非洲': 315},
   'nowConfirm': 67490,
   'rate': 191,
   'range': '03/09-03/15'},
  {'date': '03/22',
   'statis': {'亚洲': 41064,
    '其他'

In [115]:
data4.keys()

dict_keys(['FAutoContinentStatis', 'FAutoCountryConfirmAdd', 'FAutoGlobalDailyList', 'FAutoGlobalStatis'])

In [116]:
data4['FAutoContinentStatis'] # 各大洲历史每七天的数据统计
# 由于数据都需要，因此这里只要把statis弹出再追加进去就可以直接转成DataFrame，但是这样会修改原数据集，因此也可以先将该数据copy一份

[{'date': '02/16',
  'statis': {'亚洲': 247, '其他': 355, '北美洲': 23, '大洋洲': 15, '欧洲': 47},
  'nowConfirm': 600,
  'rate': 0,
  'range': ''},
 {'date': '02/23',
  'statis': {'亚洲': 969, '其他': 691, '北美洲': 42, '大洋洲': 17, '欧洲': 176},
  'nowConfirm': 1718,
  'rate': 186,
  'range': '02/17-02/23'},
 {'date': '03/01',
  'statis': {'亚洲': 4938,
   '其他': 712,
   '北美洲': 94,
   '南美洲': 3,
   '大洋洲': 29,
   '欧洲': 1564,
   '非洲': 2},
  'nowConfirm': 6210,
  'rate': 261,
  'range': '02/24-03/01'},
 {'date': '03/08',
  'statis': {'亚洲': 15145,
   '其他': 696,
   '北美洲': 528,
   '南美洲': 55,
   '大洋洲': 81,
   '欧洲': 10373,
   '非洲': 81},
  'nowConfirm': 23190,
  'rate': 273,
  'range': '03/02-03/08'},
 {'date': '03/15',
  'statis': {'亚洲': 25684,
   '其他': 712,
   '北美洲': 3315,
   '南美洲': 325,
   '大洋洲': 280,
   '欧洲': 48928,
   '非洲': 315},
  'nowConfirm': 67490,
  'rate': 191,
  'range': '03/09-03/15'},
 {'date': '03/22',
  'statis': {'亚洲': 41064,
   '其他': 712,
   '北美洲': 29104,
   '南美洲': 3213,
   '大洋洲': 1422,
   '欧洲': 15854

In [125]:
for i in data4['FAutoContinentStatis']:
    i.update(i.pop('statis')) # 把statis弹出再追加进去就可以直接转成DataFrame
data4['FAutoContinentStatis']

[{'date': '02/16',
  'nowConfirm': 600,
  'rate': 0,
  'range': '',
  '亚洲': 247,
  '其他': 355,
  '北美洲': 23,
  '大洋洲': 15,
  '欧洲': 47},
 {'date': '02/23',
  'nowConfirm': 1718,
  'rate': 186,
  'range': '02/17-02/23',
  '亚洲': 969,
  '其他': 691,
  '北美洲': 42,
  '大洋洲': 17,
  '欧洲': 176},
 {'date': '03/01',
  'nowConfirm': 6210,
  'rate': 261,
  'range': '02/24-03/01',
  '亚洲': 4938,
  '其他': 712,
  '北美洲': 94,
  '南美洲': 3,
  '大洋洲': 29,
  '欧洲': 1564,
  '非洲': 2},
 {'date': '03/08',
  'nowConfirm': 23190,
  'rate': 273,
  'range': '03/02-03/08',
  '亚洲': 15145,
  '其他': 696,
  '北美洲': 528,
  '南美洲': 55,
  '大洋洲': 81,
  '欧洲': 10373,
  '非洲': 81},
 {'date': '03/15',
  'nowConfirm': 67490,
  'rate': 191,
  'range': '03/09-03/15',
  '亚洲': 25684,
  '其他': 712,
  '北美洲': 3315,
  '南美洲': 325,
  '大洋洲': 280,
  '欧洲': 48928,
  '非洲': 315},
 {'date': '03/22',
  'nowConfirm': 201182,
  'rate': 198,
  'range': '03/16-03/22',
  '亚洲': 41064,
  '其他': 712,
  '北美洲': 29104,
  '南美洲': 3213,
  '大洋洲': 1422,
  '欧洲': 158542,
  '非洲': 12

In [126]:
pd.DataFrame(data4['FAutoContinentStatis']) # 各大洲历史每七天的数据统计

,date,nowConfirm,rate,range,亚洲,其他,北美洲,大洋洲,欧洲,南美洲,非洲
0,02/16,600,0,,247,355,23,15,47,NaN,NaN
1,02/23,1718,186,02/17-02/23,969,691,42,17,176,NaN,NaN
2,03/01,6210,261,02/24-03/01,4938,712,94,29,1564,3.0,2.0
3,03/08,23190,273,03/02-03/08,15145,696,528,81,10373,55.0,81.0
4,03/15,67490,191,03/09-03/15,25684,712,3315,280,48928,325.0,315.0
5,03/22,201182,198,03/16-03/22,41064,712,29104,1422,158542,3213.0,1249.0
6,03/29,503407,150,03/23-03/29,77597,712,133841,4489,368835,10256.0,4397.0
7,04/05,904832,79,03/30-04/05,135849,712,333668,6739,634191,23510.0,8752.0
8,04/12,1274682,40,04/06-04/12,207968,712,566955,7663,871385,48376.0,13791.0
9,04/19,1587047,24,04/13-04/19,292679,712,789241,8002,1081236,78458.0,21275.0


In [127]:
df_to_csv(pd.DataFrame(data4['FAutoContinentStatis']), filepath2, '各大洲历史每七天的数据统计.csv')

In [117]:
data4['FAutoCountryConfirmAdd']

{'不丹': 2,
 '东帝汶': 0,
 '中非共和国': 29,
 '丹麦': 91,
 '乌克兰': 638,
 '乌兹别克斯坦': 505,
 '乌干达': 4,
 '乌拉圭': 1,
 '乍得': 6,
 '也门': 76,
 '亚美尼亚': 182,
 '以色列': 457,
 '伊拉克': 2312,
 '伊朗': 2349,
 '伯利兹': 0,
 '佛得角': 75,
 '俄罗斯': 6511,
 '保加利亚': 77,
 '克罗地亚': 53,
 '冈比亚': 0,
 '冰岛': 4,
 '几内亚': 97,
 '几内亚比绍': 0,
 '列支敦士登公国': 0,
 '刚果（布）': 0,
 '刚果（金）': 42,
 '利比亚': 44,
 '利比里亚': 12,
 '加拿大': 198,
 '加纳': 270,
 '加蓬': 0,
 '匈牙利': 13,
 '北马其顿': 136,
 '南非': 12058,
 '博茨瓦纳': 85,
 '卡塔尔': 470,
 '卢旺达': 38,
 '卢森堡': 83,
 '印度': 28701,
 '印度尼西亚': 1282,
 '危地马拉': 757,
 '厄瓜多尔': 661,
 '厄立特里亚': 0,
 '叙利亚': 23,
 '古巴': 6,
 '吉尔吉斯斯坦': 488,
 '吉布提': 4,
 '哈萨克斯坦': 1646,
 '哥伦比亚': 4586,
 '哥斯达黎加': 365,
 '喀麦隆': 257,
 '土耳其': 1012,
 '圣卢西亚': 0,
 '圣基茨和尼维斯': 0,
 '圣文森特和格林纳丁斯': 6,
 '圣马力诺': 0,
 '圭亚那': 6,
 '坦桑尼亚': 0,
 '埃及': 912,
 '埃塞俄比亚': 158,
 '塔吉克斯坦': 46,
 '塞内加尔': 63,
 '塞尔维亚': 287,
 '塞拉利昂': 17,
 '塞浦路斯': 7,
 '塞舌尔': 0,
 '墨西哥': 4482,
 '多哥': 10,
 '多米尼克': 0,
 '多米尼加': 2617,
 '奥地利': 51,
 '委内瑞拉': 287,
 '孟加拉': 3099,
 '安哥拉': 23,
 '安提瓜和巴布达': 0,
 '安道尔': 0,
 '尼加拉瓜': 0,
 '尼日利亚':

In [120]:
data4['FAutoGlobalDailyList']

[{'date': '01.28',
  'all': {'confirm': 57,
   'dead': 0,
   'heal': 3,
   'newAddConfirm': 0,
   'deadRate': '0.00',
   'healRate': '5.26'}},
 {'date': '01.29',
  'all': {'confirm': 74,
   'dead': 0,
   'heal': 3,
   'newAddConfirm': 13,
   'deadRate': '0.00',
   'healRate': '4.05'}},
 {'date': '01.30',
  'all': {'confirm': 98,
   'dead': 0,
   'heal': 6,
   'newAddConfirm': 22,
   'deadRate': '0.00',
   'healRate': '6.12'}},
 {'date': '01.31',
  'all': {'confirm': 124,
   'dead': 0,
   'heal': 11,
   'newAddConfirm': 20,
   'deadRate': '0.00',
   'healRate': '8.87'}},
 {'date': '02.01',
  'all': {'confirm': 139,
   'dead': 0,
   'heal': 11,
   'newAddConfirm': 13,
   'deadRate': '0.00',
   'healRate': '7.91'}},
 {'date': '02.02',
  'all': {'confirm': 149,
   'dead': 1,
   'heal': 11,
   'newAddConfirm': 10,
   'deadRate': '0.67',
   'healRate': '7.38'}},
 {'date': '02.03',
  'all': {'confirm': 154,
   'dead': 1,
   'heal': 12,
   'newAddConfirm': 5,
   'deadRate': '0.65',
   'healRat

In [121]:
# data4['FAutoGlobalDailyList'] # 全球历史每日数据
global_daily_data = [] # 存储全球每日数据的集合
for i in data4['FAutoGlobalDailyList']:
    day_data = {'date': i['date']}
    day_data.update(i['all']) # 就all字段中的每日数据追加进去
    global_daily_data.append(day_data)
# pd.DataFrame(global_daily_data) 
df_to_csv(pd.DataFrame(global_daily_data), filepath2, '全球历史每日数据2.csv') # 转换成DataFrame，并存储到本地

In [133]:
data4['FAutoGlobalDailyList']

[{'date': '01.28',
  'all': {'confirm': 57,
   'dead': 0,
   'heal': 3,
   'newAddConfirm': 0,
   'deadRate': '0.00',
   'healRate': '5.26'}},
 {'date': '01.29',
  'all': {'confirm': 74,
   'dead': 0,
   'heal': 3,
   'newAddConfirm': 13,
   'deadRate': '0.00',
   'healRate': '4.05'}},
 {'date': '01.30',
  'all': {'confirm': 98,
   'dead': 0,
   'heal': 6,
   'newAddConfirm': 22,
   'deadRate': '0.00',
   'healRate': '6.12'}},
 {'date': '01.31',
  'all': {'confirm': 124,
   'dead': 0,
   'heal': 11,
   'newAddConfirm': 20,
   'deadRate': '0.00',
   'healRate': '8.87'}},
 {'date': '02.01',
  'all': {'confirm': 139,
   'dead': 0,
   'heal': 11,
   'newAddConfirm': 13,
   'deadRate': '0.00',
   'healRate': '7.91'}},
 {'date': '02.02',
  'all': {'confirm': 149,
   'dead': 1,
   'heal': 11,
   'newAddConfirm': 10,
   'deadRate': '0.67',
   'healRate': '7.38'}},
 {'date': '02.03',
  'all': {'confirm': 154,
   'dead': 1,
   'heal': 12,
   'newAddConfirm': 5,
   'deadRate': '0.65',
   'healRat

# 外国疫情按国家查看

In [130]:
url5 = "https://api.inews.qq.com/newsqa/v1/automation/foreign/country/ranklist"
res5 = requests.get(url5, headers=headers2)
data5 = res5.json()['data']
data5

[{'name': '美国',
  'continent': '北美洲',
  'date': '07.13',
  'isUpdated': True,
  'confirmAdd': 58349,
  'confirmAddCut': 0,
  'confirm': 3413995,
  'suspect': 0,
  'dead': 137782,
  'heal': 1517084,
  'nowConfirm': 1759129,
  'confirmCompare': 58349,
  'nowConfirmCompare': 30305,
  'healCompare': 27664,
  'deadCompare': 380},
 {'name': '巴西',
  'continent': '南美洲',
  'date': '07.13',
  'isUpdated': False,
  'confirmAdd': 24831,
  'confirmAddCut': 0,
  'confirm': 1864681,
  'suspect': 0,
  'dead': 72100,
  'heal': 1264843,
  'nowConfirm': 527738,
  'confirmCompare': 24831,
  'nowConfirmCompare': 3445,
  'healCompare': 20755,
  'deadCompare': 631},
 {'name': '印度',
  'continent': '亚洲',
  'date': '07.13',
  'isUpdated': False,
  'confirmAdd': 28701,
  'confirmAddCut': 0,
  'confirm': 878254,
  'suspect': 0,
  'dead': 23174,
  'heal': 553471,
  'nowConfirm': 301609,
  'confirmCompare': 28701,
  'nowConfirmCompare': 9351,
  'healCompare': 18850,
  'deadCompare': 500},
 {'name': '俄罗斯',
  'contin

In [131]:
pd.DataFrame(data5) # 可直接转DataFrame

,name,continent,date,isUpdated,confirmAdd,confirmAddCut,confirm,suspect,dead,heal,nowConfirm,confirmCompare,nowConfirmCompare,healCompare,deadCompare
0,美国,北美洲,07.13,True,58349,0,3413995,0,137782,1517084,1759129,58349,30305,27664,380
1,巴西,南美洲,07.13,False,24831,0,1864681,0,72100,1264843,527738,24831,3445,20755,631
2,印度,亚洲,07.13,False,28701,0,878254,0,23174,553471,301609,28701,9351,18850,500
3,俄罗斯,欧洲,07.13,False,6511,0,732547,0,11422,503168,217957,6511,3447,2960,104
4,秘鲁,南美洲,07.13,False,3616,0,326326,0,11870,217111,97345,3616,469,2959,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,老挝,亚洲,07.13,False,0,0,19,0,0,19,0,0,0,0,0
178,多米尼克,北美洲,07.13,False,0,0,18,0,0,18,0,0,0,0,0
179,圣基茨和尼维斯,北美洲,07.13,False,0,0,17,0,0,15,2,0,0,0,0
180,梵蒂冈,欧洲,07.13,False,0,0,12,0,0,12,0,0,0,0,0


In [132]:
# 存储
df_to_csv(pd.DataFrame(data5), filepath2, '外国疫情数据（按国家查看）.csv')

# 外国部分国家数据（随时间变化）

In [2]:
url6 = "https://api.inews.qq.com/newsqa/v1/automation/modules/list?modules=FAutoCountryMerge"
res6 = requests.get(url6, headers=headers2)
data6 = res6.json()['data']

In [3]:
data6

{'FAutoCountryMerge': {'俄罗斯': {'showDash': False,
   'list': [{'date': '01.31', 'confirm': 2},
    {'date': '02.01', 'confirm': 2},
    {'date': '02.02', 'confirm': 2},
    {'date': '02.03', 'confirm': 2},
    {'date': '02.04', 'confirm': 2},
    {'date': '02.05', 'confirm': 2},
    {'date': '02.06', 'confirm': 2},
    {'date': '02.07', 'confirm': 2},
    {'date': '02.08', 'confirm': 2},
    {'date': '02.09', 'confirm': 2},
    {'date': '02.10', 'confirm': 2},
    {'date': '02.11', 'confirm': 2},
    {'date': '02.12', 'confirm': 2},
    {'date': '02.13', 'confirm': 2},
    {'date': '02.14', 'confirm': 2},
    {'date': '02.15', 'confirm': 2},
    {'date': '02.16', 'confirm': 2},
    {'date': '02.17', 'confirm': 2},
    {'date': '02.18', 'confirm': 2},
    {'date': '02.19', 'confirm': 2},
    {'date': '02.20', 'confirm': 2},
    {'date': '02.21', 'confirm': 2},
    {'date': '02.22', 'confirm': 2},
    {'date': '02.23', 'confirm': 2},
    {'date': '02.24', 'confirm': 2},
    {'date': '02.

In [4]:
data6.keys()

dict_keys(['FAutoCountryMerge'])

In [6]:
data6['FAutoCountryMerge'].keys()

dict_keys(['俄罗斯', '巴西', '德国', '意大利', '法国', '美国', '英国', '西班牙'])

In [12]:
part_coutnries = data6['FAutoCountryMerge'].keys()
for i in part_countries: # i 为FAutoCountryMerge中的key，即每个国家的国家名
    country_data = data6['FAutoCountryMerge'][i]
    pd.DataFrame(country_data['list'])
    break


In [14]:
pd.DataFrame(data6['FAutoCountryMerge']['美国']['list'])

,date,confirm
0,01.28,5
1,01.29,5
2,01.30,6
3,01.31,6
4,02.01,7
...,...,...
164,07.10,3219999
165,07.11,3291786
166,07.12,3355646
167,07.13,3413995


In [23]:
url7 = "https://api.inews.qq.com/newsqa/v1/automation/foreign/daily/list?country=美国"
res7 = requests.get(url7, headers=headers2)
data7 = res7.json()
data7

{'ret': 0,
 'info': '',
 'data': [{'date': '01.28',
   'confirm_add': 0,
   'confirm': 5,
   'heal': 0,
   'dead': 0},
  {'date': '01.29', 'confirm_add': 0, 'confirm': 5, 'heal': 0, 'dead': 0},
  {'date': '01.30', 'confirm_add': 1, 'confirm': 6, 'heal': 0, 'dead': 0},
  {'date': '01.31', 'confirm_add': 0, 'confirm': 6, 'heal': 0, 'dead': 0},
  {'date': '02.01', 'confirm_add': 1, 'confirm': 7, 'heal': 0, 'dead': 0},
  {'date': '02.02', 'confirm_add': 1, 'confirm': 8, 'heal': 0, 'dead': 0},
  {'date': '02.03', 'confirm_add': 3, 'confirm': 11, 'heal': 1, 'dead': 0},
  {'date': '02.04', 'confirm_add': 0, 'confirm': 11, 'heal': 1, 'dead': 0},
  {'date': '02.05', 'confirm_add': 0, 'confirm': 11, 'heal': 1, 'dead': 0},
  {'date': '02.06', 'confirm_add': 1, 'confirm': 12, 'heal': 1, 'dead': 0},
  {'date': '02.07', 'confirm_add': 0, 'confirm': 12, 'heal': 1, 'dead': 0},
  {'date': '02.08', 'confirm_add': 0, 'confirm': 12, 'heal': 1, 'dead': 0},
  {'date': '02.09', 'confirm_add': 0, 'confirm': 1

In [27]:
if data7['data']:
    print("hahha")
if abc:
    print("abc")
else:
    print("cba")

hahha
cba
